In [1]:
from osgeo import gdal
import sys
import os
import pandas as pd

In [2]:
def get_extend(file_path):
    src = gdal.Open(file_path)
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
    lrx = ulx + (src.RasterXSize * xres)
    lry = uly + (src.RasterYSize * yres)
    return(ulx,uly,lrx,lry)

In [3]:
ulx,uly,lrx,lry = get_extend('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/dtm/25-46.tif')

print('upper left corner: ', ulx, uly)
print('lower right corner: ', lrx, lry)

upper left corner:  173462.08030918773 389578.5985227945
lower right corner:  175027.55069879314 387762.213018567


## get info

In [5]:
src = gdal.Open('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/dtm/25-46.tif')

print("Driver: {}/{}".format(src.GetDriver().ShortName,
                            src.GetDriver().LongName))
print("Size is {} x {} x {}".format(src.RasterXSize,
                                    src.RasterYSize,
                                    src.RasterCount))
print("Projection is {}".format(src.GetProjection()))
geotransform = src.GetGeoTransform()
if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

Driver: GTiff/GeoTIFF
Size is 1566 x 1817 x 1
Projection is PROJCS["Austria_Lambert",GEOGCS["GCS_BESSEL_AUT",DATUM["D_BESSEL_AUT",SPHEROID["Bessel_1841",6377397.155,299.1528128,AUTHORITY["EPSG","7004"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",47.5],PARAMETER["central_meridian",13.333333333],PARAMETER["standard_parallel_1",46],PARAMETER["standard_parallel_2",49],PARAMETER["false_easting",400000],PARAMETER["false_northing",400000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Origin = (173462.08030918773, 389578.5985227945)
Pixel Size = (0.9996618069, -0.9996618075)


In [14]:
cd = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/dtm/'
directory = os.fsencode(cd)

extend = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".tif"): 
        #print(os.path.join(directory, filename))
        print('#############################################################')
        print(filename)
        ulx,uly,lrx,lry = get_extend(cd + filename)
        print('upper left corner: ', ulx, uly)
        print('lower right corner: ', lrx, lry)
        extend.append([filename, ulx, uly, lrx, lry])

#############################################################
22-37.tif
upper left corner:  165027.2994244552 367762.7905738257
lower right corner:  167527.48423100822 365262.6057642715
#############################################################
25-29.tif
upper left corner:  172527.44882720403 347762.93837321247
lower right corner:  175027.69326885152 345262.6939318151
#############################################################
21-42.tif
upper left corner:  162527.56106065083 380262.85326321574
lower right corner:  165026.72122865083 377762.69343039824
#############################################################
24-30.tif
upper left corner:  170027.46188570355 350262.0797642789
lower right corner:  172527.70103573526 347762.8403092789
#############################################################
23-44.tif
upper left corner:  168274.55322486328 384776.9458346535
lower right corner:  170026.96426559667 382762.62282024854
#############################################################


In [15]:
extend

[['22-37.tif',
  165027.2994244552,
  367762.7905738257,
  167527.48423100822,
  365262.6057642715],
 ['25-29.tif',
  172527.44882720403,
  347762.93837321247,
  175027.69326885152,
  345262.6939318151],
 ['21-42.tif',
  162527.56106065083,
  380262.85326321574,
  165026.72122865083,
  377762.69343039824],
 ['24-30.tif',
  170027.46188570355,
  350262.0797642789,
  172527.70103573526,
  347762.8403092789],
 ['23-44.tif',
  168274.55322486328,
  384776.9458346535,
  170026.96426559667,
  382762.62282024854],
 ['20-35.tif',
  160027.58833882774,
  362762.82373764017,
  162526.79163907774,
  360262.62075581995],
 ['23-38.tif',
  167527.1330577352,
  370262.59521025047,
  170027.31355806638,
  367762.41471016937],
 ['25-41.tif',
  172527.26714307308,
  377762.5182568829,
  175027.43472223787,
  375262.35067471693],
 ['23-42.tif',
  167527.1610342969,
  380262.8296516058,
  170027.3235434345,
  377762.667141968],
 ['25-39.tif',
  172526.7745390213,
  372762.76275494904,
  175026.9561833099,

In [20]:
df = pd.DataFrame(extend)
df.columns =['file','ulx','uly','lrx','lry']

In [21]:
df

,file,ulx,uly,lrx,lry
0,22-37.tif,165027.299424,367762.790574,167527.484231,365262.605764
1,25-29.tif,172527.448827,347762.938373,175027.693269,345262.693932
2,21-42.tif,162527.561061,380262.853263,165026.721229,377762.693430
3,24-30.tif,170027.461886,350262.079764,172527.701036,347762.840309
4,23-44.tif,168274.553225,384776.945835,170026.964266,382762.622820
...,...,...,...,...,...
165,21-37.tif,162526.975939,367762.313059,165027.159218,365262.129775
166,18-23.tif,155026.963792,332762.508096,157527.272694,330262.199194
167,24-44.tif,170026.964266,384776.945835,172527.123999,382762.622820
168,21-27.tif,162527.316158,342762.080686,165027.586107,340262.810449


In [ ]:
ulx, uly, lrx, lry

In [22]:
ulx

155026.98438577197

In [28]:
e_ulx = 168500
#e_uly = 355580
e_lrx = 173888
#e_lry = 351200

e_lry = 355580
e_uly = 351200

In [29]:
df[(df['ulx'] > e_ulx) & (df['lrx'] < e_lrx) & (df['uly'] > e_uly) & (df['lry'] < e_lry)]

,file,ulx,uly,lrx,lry
12,24-33.tif,170027.077222,357762.906874,172527.289943,355262.694152
60,24-32.tif,170026.959540,355262.638880,172527.192498,352762.405920
105,24-31.tif,170026.959540,352762.405920,172527.192498,350262.172960


In [6]:
## mosaik two images

In [7]:
# mosaic two images together
import os, sys, gdal
from gdalconst import *

In [8]:
def mosaic(path1, path2, pathout):
    
    # register all of the GDAL drivers
    gdal.AllRegister()

    # read in doq1 and get info about it
    ds1 = gdal.Open(path1)
    band1 = ds1.GetRasterBand(1)
    rows1 = ds1.RasterYSize
    cols1 = ds1.RasterXSize

    # get the corner coordinates for doq1
    transform1 = ds1.GetGeoTransform()
    minX1 = transform1[0]
    maxY1 = transform1[3]
    pixelWidth1 = transform1[1]
    pixelHeight1 = transform1[5]
    maxX1 = minX1 + (cols1 * pixelWidth1)
    minY1 = maxY1 + (rows1 * pixelHeight1)

    # read in doq2 and get info about it
    ds2 = gdal.Open(path2)
    band2 = ds2.GetRasterBand(1)
    rows2 = ds2.RasterYSize
    cols2 = ds2.RasterXSize

    # get the corner coordinates for doq2
    transform2 = ds2.GetGeoTransform()
    minX2 = transform2[0]
    maxY2 = transform2[3]
    pixelWidth2 = transform2[1]
    pixelHeight2 = transform2[5]
    maxX2 = minX2 + (cols2 * pixelWidth2)
    minY2 = maxY2 + (rows2 * pixelHeight2)

    # get the corner coordinates for the output
    minX = min(minX1, minX2)
    maxX = max(maxX1, maxX2)
    minY = min(minY1, minY2)
    maxY = max(maxY1, maxY2)

    # get the number of rows and columns for the output
    cols = int((maxX - minX) / pixelWidth1)
    rows = int((maxY - minY) / abs(pixelHeight1))

    # compute the origin (upper left) offset for doq1
    xOffset1 = int((minX1 - minX) / pixelWidth1)
    yOffset1 = int((maxY1 - maxY) / pixelHeight1)

    # compute the origin (upper left) offset for doq2
    xOffset2 = int((minX2 - minX) / pixelWidth1)
    yOffset2 = int((maxY2 - maxY) / pixelHeight1)

    # create the output image
    driver = ds1.GetDriver()
    dsOut = driver.Create(pathout, cols, rows, 1, band1.DataType)
    bandOut = dsOut.GetRasterBand(1)

    # read in doq1 and write it to the output
    data1 = band1.ReadAsArray(0, 0, cols1, rows1)
    bandOut.WriteArray(data1, xOffset1, yOffset1)

    # read in doq2 and write it to the output
    data2 = band2.ReadAsArray(0, 0, cols2, rows2)
    bandOut.WriteArray(data2, xOffset2, yOffset2)

    # compute statistics for the output
    bandOut.FlushCache()
    stats = bandOut.GetStatistics(0, 1)

    # set the geotransform and projection on the output
    geotransform = [minX, pixelWidth1, 0, maxY, 0, pixelHeight1]
    dsOut.SetGeoTransform(geotransform)
    dsOut.SetProjection(ds1.GetProjection())

    # build pyramids for the output
    #gdal.SetConfigOption('HFA_USE_RRD', 'YES')
    #dsOut.BuildOverviews(overviewlist=[2,4,8,16])

In [9]:
path1 = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/dtm/24-43.tif'
path2 = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/dtm/25-43.tif'
pathout = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/dtm/out.tif'

mosaic(path1, path2, pathout)